In [1]:
%cd C:/Users/sylee/Documents/GitHub/Final_2Kyoungh/data

C:\Users\sylee\Documents\GitHub\Final_2Kyoungh\data


In [2]:
import requests
import json
import pandas as pd

## 기본 세팅 부분
##종웅님 
headers = {"X-Naver-Client-Id": "EmdlUXsc4CuKNm_kxYKS",
           "X-Naver-Client-Secret":"dSLoIEmMgy"}

url = "https://openapi.naver.com/v1/datalab/shopping/categories"
def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df


In [3]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
          {"name": "남성화장품", "param": [ "50000202"]},
          {"name": "네일케어", "param": [ "50000196"]},
          {"name": "바디케어", "param": [ "50000197"]},
          {"name": "베이스메이크업", "param": [ "50000194"]},
          {"name": "뷰티소품", "param": [ "50000201"]},
          {"name": "색조메이크업", "param": [ "50000195"]},
          {"name": "선케어", "param": [ "50000191"]},
          {"name": "클렌징", "param": [ "50000192"]},
          {"name": "향수", "param": [ "50000200"]},
          {"name": "헤어케어", "param": [ "50000198"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "화장품/미용", "param": [ "50000002"]},
      {"name": "스킨케어", "param": [ "50000190"]},
    ],
    "device":i,
    "gender":"",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "화장품/미용")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "스킨케어")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='전체'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='전체'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='전체'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_a=pd.concat(df_list)
df_all_a.drop(["화장품/미용"], axis=1, inplace=True)
df_all_a

,스킨케어,남성화장품,네일케어,바디케어,베이스메이크업,뷰티소품,색조메이크업,선케어,클렌징,향수,헤어케어,유형,성별,연령
period,,,,,,,,,,,,,,
2021-05-10,6.64098,0.73318,0.70179,3.28160,1.97185,1.06024,2.37850,1.38640,1.71647,1.64940,2.95931,PC+MO,전체,전체
2021-05-11,6.22457,0.70120,0.69326,3.12894,1.99454,0.99269,2.39604,1.38054,1.65699,1.68635,2.59959,PC+MO,전체,전체
2021-05-12,5.85114,0.67180,0.67996,2.90579,1.89944,0.98827,2.34788,1.42383,1.58860,1.61227,2.57453,PC+MO,전체,전체
2021-05-13,5.23610,0.62774,0.68538,2.76543,1.90595,0.96612,2.13089,1.52143,1.63042,1.54362,2.35832,PC+MO,전체,전체
2021-05-14,4.86541,0.53362,0.62259,2.61945,1.66118,0.89539,2.03453,1.47911,1.58780,1.62502,2.20560,PC+MO,전체,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,3.94525,0.39053,0.65205,2.06157,1.81792,0.96923,2.00520,1.03155,1.15490,1.27793,2.12259,MO,전체,전체
2023-06-30,3.48118,0.35935,0.60796,1.74881,1.62001,0.86788,1.87364,0.86894,1.03057,1.14260,1.92038,MO,전체,전체
2023-07-01,3.99609,0.42167,0.61510,2.07903,1.75047,0.92293,2.17475,1.17509,1.24927,1.37330,2.30965,MO,전체,전체


In [4]:
for i in devices:
    categories = [
          {"name": "남성화장품", "param": [ "50000202"]},
          {"name": "네일케어", "param": [ "50000196"]},
          {"name": "바디케어", "param": [ "50000197"]},
          {"name": "베이스메이크업", "param": [ "50000194"]},
          {"name": "뷰티소품", "param": [ "50000201"]},
          {"name": "색조메이크업", "param": [ "50000195"]},
          {"name": "선케어", "param": [ "50000191"]},
          {"name": "클렌징", "param": [ "50000192"]},
          {"name": "향수", "param": [ "50000200"]},
          {"name": "헤어케어", "param": [ "50000198"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "화장품/미용", "param": [ "50000002"]},
      {"name": "스킨케어", "param": [ "50000190"]},
    ],
    "device":i,
    "gender":"f",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "화장품/미용")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "스킨케어")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='여성'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='여성'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='여성'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_b=pd.concat(df_list)
df_all_b.drop(["화장품/미용"], axis=1, inplace=True)
df_all_b

,스킨케어,남성화장품,네일케어,바디케어,베이스메이크업,뷰티소품,색조메이크업,선케어,클렌징,향수,헤어케어,유형,성별,연령
period,,,,,,,,,,,,,,
2021-05-10,11.40664,0.61651,1.49373,5.18402,4.01541,2.09775,4.94097,2.37154,2.78559,2.72305,4.55130,PC+MO,여성,전체
2021-05-11,10.53975,0.58947,1.44619,4.89885,4.07537,1.94437,4.96951,2.37288,2.59205,2.70947,4.02920,PC+MO,여성,전체
2021-05-12,9.97220,0.52555,1.44427,4.59976,3.83441,1.93695,4.86201,2.42617,2.62705,2.48005,3.99078,PC+MO,여성,전체
2021-05-13,8.67505,0.49513,1.45873,4.23487,3.86162,1.91124,4.34563,2.63976,2.40742,2.41025,3.57553,PC+MO,여성,전체
2021-05-14,7.88674,0.41992,1.31598,3.90237,3.29865,1.75441,4.18091,2.60401,2.04508,2.61534,3.31319,PC+MO,여성,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,8.10137,0.35609,1.51194,3.87390,4.33478,2.07646,4.81647,2.08340,2.24706,2.22951,4.06227,MO,여성,전체
2023-06-30,7.14019,0.31815,1.42599,3.25778,3.87014,1.84224,4.48694,1.74955,1.97735,1.99173,3.61485,MO,여성,전체
2023-07-01,8.15437,0.35131,1.44202,3.84780,4.10022,1.96296,5.19321,2.31200,2.41200,2.39753,4.33525,MO,여성,전체


In [5]:
for i in devices:
    categories = [
          {"name": "남성화장품", "param": [ "50000202"]},
          {"name": "네일케어", "param": [ "50000196"]},
          {"name": "바디케어", "param": [ "50000197"]},
          {"name": "베이스메이크업", "param": [ "50000194"]},
          {"name": "뷰티소품", "param": [ "50000201"]},
          {"name": "색조메이크업", "param": [ "50000195"]},
          {"name": "선케어", "param": [ "50000191"]},
          {"name": "클렌징", "param": [ "50000192"]},
          {"name": "향수", "param": [ "50000200"]},
          {"name": "헤어케어", "param": [ "50000198"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "화장품/미용", "param": [ "50000002"]},
      {"name": "스킨케어", "param": [ "50000190"]},
    ],
    "device":i,
    "gender":"m",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "화장품/미용")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "스킨케어")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='남성'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='남성'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='남성'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_c=pd.concat(df_list)
df_all_c.drop(["화장품/미용"], axis=1, inplace=True)
df_all_c

,스킨케어,남성화장품,네일케어,바디케어,베이스메이크업,뷰티소품,색조메이크업,선케어,클렌징,향수,헤어케어,유형,성별,연령
period,,,,,,,,,,,,,,
2021-05-10,13.91708,5.37857,0.93509,9.23403,1.55481,2.48968,1.80541,3.04715,4.03025,5.66638,8.69424,PC+MO,남성,전체
2021-05-11,12.35078,4.99544,0.92935,8.52403,1.53138,2.40744,1.74776,2.84525,3.90116,5.88528,6.54221,PC+MO,남성,전체
2021-05-12,11.43842,4.71107,0.89076,8.22198,1.45191,2.37000,1.71284,3.03130,3.75622,5.47711,6.26658,PC+MO,남성,전체
2021-05-13,10.41742,4.29165,0.85493,7.62707,1.47557,2.23839,1.49808,3.44269,3.44154,5.02966,5.61608,PC+MO,남성,전체
2021-05-14,9.10149,3.55179,0.78556,6.83140,1.25735,2.07415,1.37634,2.97801,2.89670,4.97959,5.05447,PC+MO,남성,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,10.48977,4.00537,0.86545,7.92953,1.68636,2.47086,1.74938,3.17143,3.59601,5.92834,6.99399,MO,남성,전체
2023-06-30,9.29291,3.71281,0.70898,6.91875,1.49485,2.36030,1.67603,2.74495,3.19886,5.60101,6.61423,MO,남성,전체
2023-07-01,10.94613,4.45847,0.74077,8.29679,1.83304,2.44397,1.92703,3.95240,3.85760,6.81742,8.17347,MO,남성,전체


In [6]:
df_list=[df_all_a,df_all_b,df_all_c]
df_all=pd.concat(df_list)

## 엑셀에 저장
df_all.to_excel("MCLS_11_all_fm_all.xlsx", sheet_name="click ratio")
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7065 entries, 2021-05-10 to 2023-07-03
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   스킨케어     7065 non-null   float64
 1   남성화장품    7065 non-null   float64
 2   네일케어     7065 non-null   float64
 3   바디케어     7065 non-null   float64
 4   베이스메이크업  7065 non-null   float64
 5   뷰티소품     7065 non-null   float64
 6   색조메이크업   7065 non-null   float64
 7   선케어      7065 non-null   float64
 8   클렌징      7065 non-null   float64
 9   향수       7065 non-null   float64
 10  헤어케어     7065 non-null   float64
 11  유형       7065 non-null   object 
 12  성별       7065 non-null   object 
 13  연령       7065 non-null   object 
dtypes: float64(11), object(3)
memory usage: 827.9+ KB
